In [39]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100) 

In [40]:
file_path = "TushyCartonisationAnalysis_recreate.xlsx"
excel_sheets = pd.ExcelFile(file_path).sheet_names

In [41]:
invoice_df = pd.read_excel(file_path, sheet_name='Raw Baja Invoices')

invoice_df['Charge'] = invoice_df['Charge'].str.upper()
invoice_df_drop_columns = ['Reference','Entry Date', 'Entry Number', 'Customs Value Currency']
invoice_df.drop(columns=invoice_df_drop_columns,axis=1,inplace=True)
invoice_df['id'] = range(1, len(invoice_df) + 1)

invoice_df_data_types = {"Order Number":"str"}
invoice_df = invoice_df.astype(invoice_df_data_types)
invoice_df['Transaction Date'] = pd.to_datetime(invoice_df['Transaction Date'], unit='D', origin='1899-12-30').dt.strftime('%Y-%m-%d')

In [42]:
invoice_df.columns

Index(['Invoice', 'Invoice Date', 'Order ID', 'Order Number', 'Profile',
       'Carrier', 'Transaction Date', 'Tracking Number', 'Service Type',
       'Charge', 'Package Quantity', 'Packaging Type', 'Entered Weight (LB)',
       'Billed Weight (LB)', 'Dim Length', 'Dim Width', 'Dim Height', 'Zone',
       'Origin State', 'Origin Country', 'Receiver Name', 'Receiver Company',
       'Receiver Address Line 1', 'Receiver Address Line 2', 'Receiver City',
       'Receiver State', 'Receiver Zip', 'Receiver Country', 'Customs Value',
       'Duty & Taxes', 'Bill Amount', 'id'],
      dtype='object')

In [43]:
invoice_df.dtypes

Invoice                             int64
Invoice Date               datetime64[ns]
Order ID                            int64
Order Number                       object
Profile                            object
Carrier                            object
Transaction Date                   object
Tracking Number                    object
Service Type                       object
Charge                             object
Package Quantity                  float64
Packaging Type                     object
Entered Weight (LB)               float64
Billed Weight (LB)                float64
Dim Length                        float64
Dim Width                         float64
Dim Height                        float64
Zone                               object
Origin State                       object
Origin Country                     object
Receiver Name                      object
Receiver Company                   object
Receiver Address Line 1            object
Receiver Address Line 2           

In [44]:
invoice_df.head()

,Invoice,Invoice Date,Order ID,Order Number,Profile,Carrier,Transaction Date,Tracking Number,Service Type,Charge,Package Quantity,Packaging Type,Entered Weight (LB),Billed Weight (LB),Dim Length,Dim Width,Dim Height,Zone,Origin State,Origin Country,Receiver Name,Receiver Company,Receiver Address Line 1,Receiver Address Line 2,Receiver City,Receiver State,Receiver Zip,Receiver Country,Customs Value,Duty & Taxes,Bill Amount,id
0,33477,2025-03-20,576899052,#1442861,default,UPS,2025-02-11,1ZAC8375YW34252402,UPS SurePost - 1 lb or Greater,TRANSPORTATION,1.0,PKG,4.1,5.0,19.0,9.0,5.0,7,TX,US,Dana Wallace,"TUSHY, Inc",10813 NW 30TH ST STE 115,NaN,DORAL,FL,331922147,US,0.0,0.0,9.80,1
1,33477,2025-03-20,576899052,#1442861,default,UPS,2025-02-11,1ZAC8375YW34252402,UPS SurePost - 1 lb or Greater,FUEL SURCHARGE,1.0,PKG,4.1,5.0,19.0,9.0,5.0,7,TX,US,Dana Wallace,"TUSHY, Inc",10813 NW 30TH ST STE 115,NaN,DORAL,FL,331922147,US,0.0,0.0,1.23,2
2,33477,2025-03-20,581683521,#1446209,default,UPS,2025-02-24,1ZAC8375YW14943702,UPS SurePost - 1 lb or Greater,TRANSPORTATION,1.0,PKG,2.1,3.0,18.0,10.0,4.0,7,TX,US,NaN,Jacob Vosper,22 DORLAND AVE,NaN,POUGHKEEPSIE,NY,126036404,US,0.0,0.0,8.90,3
3,33477,2025-03-20,581683521,#1446209,default,UPS,2025-02-24,1ZAC8375YW14943702,UPS SurePost - 1 lb or Greater,FUEL SURCHARGE,1.0,PKG,2.1,3.0,18.0,10.0,4.0,7,TX,US,NaN,Jacob Vosper,22 DORLAND AVE,NaN,POUGHKEEPSIE,NY,126036404,US,0.0,0.0,1.12,4
4,33477,2025-03-20,582604555,#1446902,default,UPS,2025-02-26,1ZAC83750301909861,Ground Residential,TRANSPORTATION,1.0,PKG,0.1,18.0,23.0,21.0,8.0,7,TX,US,NaN,Alisha Sare,277 SANTA ROSA AVE,PUSH 3 ON THE HILAVATOR OR TAKE,SAUSALITO,CA,949652036,US,0.0,0.0,16.86,5


In [45]:
charge_df = pd.read_excel(file_path, sheet_name='Charge Map')
charge_df['Charge'] = charge_df['Charge'].str.upper()
charge_map_df = charge_df[["Charge","Map"]]

In [46]:
invoice_charge_df = invoice_df.merge(charge_map_df, on='Charge', how='inner')

In [47]:
#missing Charge "Return To Sender - Web Request" and map "Returns"
invoice_charge_df["Map"].unique()

array(['Freight', 'Fuel', 'Residential', 'Surcharge', 'Freight Adj',
       'Fuel Adj', 'Surcharge Adj', 'Duties/Tax'], dtype=object)

In [48]:
invoice_Flatten = invoice_charge_df.copy()
unique_maps = invoice_Flatten['Map'].unique()
for map_value in unique_maps:
    invoice_Flatten[map_value] = invoice_Flatten.apply(
        lambda row: row['Bill Amount'] if row['Map'] == map_value else 0, axis=1
    )

In [49]:
grouped_df = invoice_Flatten.groupby(
    ['Order Number', 'Tracking Number', 'Origin State', 'Transaction Date', 'Receiver Zip', 'Carrier', 'Service Type'],
    as_index=False
).agg({
    'Freight': 'sum',
    'Fuel': 'sum',
    'Residential': 'sum',
    'Surcharge': 'sum',
    'Freight Adj': 'sum',
    'Fuel Adj': 'sum',
    'Surcharge Adj': 'sum',
    'Duties/Tax': 'sum',
    'Entered Weight (LB)': 'max',
    'Billed Weight (LB)': 'max',
    'Dim Length': 'max',
    'Dim Width': 'max',
    'Dim Height': 'max',
})
grouped_df['F+F+R'] = grouped_df['Freight'] + grouped_df['Fuel'] + grouped_df['Residential']
grouped_df['Bill Amount'] = grouped_df[['Freight', 'Freight Adj', 'Fuel', 'Fuel Adj', 'Residential', 'Surcharge', 'Surcharge Adj', 'Duties/Tax']].sum(axis=1)

In [72]:
# Step 1: Sort by Order Number and Entered Weight (Descending)
df_sorted = grouped_df.sort_values(['Order Number', 'Entered Weight (LB)'], ascending=[True, True]).reset_index(drop=True)

# Step 2: Add group_count
df_sorted['group_count'] = df_sorted.groupby('Order Number')['Order Number'].transform('count')

# Step 3: Add temp rank within each Order Number
df_sorted['temp_rank'] = df_sorted.groupby('Order Number').cumcount()

# Step 4: Assign pair_count based on the rules
pair_count_list = []
current_pair = 1

for order_number, group in df_sorted.groupby('Order Number'):
    n = len(group)
    for idx, row in group.iterrows():
        # Odd groups: Last item gets no pair_count
        if n % 2 == 1 and row['temp_rank'] == n-1:
            pair_count_list.append(None)
        else:
            if row['temp_rank'] % 2 == 0:
                pair_count_list.append(current_pair)
            else:
                pair_count_list.append(current_pair)
                current_pair += 1

df_sorted['pair_count'] = pair_count_list

# Step 5: Clean up
df_final = df_sorted.drop(columns=['temp_rank'])

In [73]:
df_final['pair_count'] = df_final['pair_count'].fillna('-')

In [74]:
df_final[df_final['Order Number'].isin(['#1435752','#1451227','#1448451','#1448934(1)','EXC-1446494-1'])]

,Order Number,Tracking Number,Origin State,Transaction Date,Receiver Zip,Carrier,Service Type,Freight,Fuel,Residential,Surcharge,Freight Adj,Fuel Adj,Surcharge Adj,Duties/Tax,Entered Weight (LB),Billed Weight (LB),Dim Length,Dim Width,Dim Height,F+F+R,Bill Amount,group_count,pair_count
22,#1435752,D10016188981916,CA,2025-03-06,30309-4130,OnTrac,C,47.41,5.54,0.88,9.25,0.0,0.0,0.0,0.0,15.00,47.0,19.0,8.0,23.0,53.83,63.08,2,1.0
23,#1435752,D10016188984671,CA,2025-03-06,30309-4130,OnTrac,C,23.84,3.28,0.88,9.25,0.0,0.0,0.0,0.0,15.00,20.0,19.0,9.0,23.0,28.00,37.25,2,1.0
954,#1448451,1ZAC83750306098167,TX,2025-03-03,294077607,UPS,Ground Residential,10.89,1.80,3.29,0.00,0.0,0.0,0.0,0.0,3.10,8.0,21.0,17.0,5.0,15.98,15.98,3,25.0
955,#1448451,1ZAC83750309295171,TX,2025-03-03,294077607,UPS,Ground Residential,10.89,1.80,3.29,0.00,0.0,0.0,0.0,0.0,3.10,8.0,21.0,17.0,5.0,15.98,15.98,3,25.0
956,#1448451,1ZAC83750319176585,TX,2025-03-03,294077607,UPS,Ground Residential,10.03,1.67,3.29,0.00,0.0,0.0,0.0,0.0,3.10,4.0,18.0,10.0,4.0,14.99,14.99,3,-
1254,#1448934(1),5101306582,WA,2025-03-07,V3X 1N8,DHL Express,EXPRESS WORLDWIDE nondoc,0.00,12.68,0.00,59.65,0.0,0.0,0.0,0.0,31.27,28.0,NaN,NaN,NaN,12.68,72.33,4,37.0
1255,#1448934(1),5101306792,WA,2025-03-07,V3X 1N8,DHL Express,EXPRESS WORLDWIDE nondoc,0.00,12.68,0.00,59.65,0.0,0.0,0.0,0.0,31.27,28.0,NaN,NaN,NaN,12.68,72.33,4,37.0
1256,#1448934(1),5101307035,WA,2025-03-07,V3X 1N8,DHL Express,EXPRESS WORLDWIDE nondoc,0.00,12.68,0.00,59.65,0.0,0.0,0.0,0.0,31.27,28.0,NaN,NaN,NaN,12.68,72.33,4,38.0
1257,#1448934(1),9067659005,WA,2025-03-07,V3X 1N8,DHL Express,EXPRESS WORLDWIDE nondoc,0.00,12.68,0.00,59.65,0.0,0.0,0.0,0.0,31.27,28.0,NaN,NaN,NaN,12.68,72.33,4,38.0
2542,#1451227,92346903255588300007010602,CA,2025-03-14,90049-3638,USPS,Ground Advantage,7.45,0.00,0.00,0.00,0.0,0.0,0.0,0.0,4.00,4.0,14.5,9.5,1.0,7.45,7.45,4,93.0


In [75]:
df_final.isna().sum()

Order Number            0
Tracking Number         0
Origin State            0
Transaction Date        0
Receiver Zip            0
Carrier                 0
Service Type            0
Freight                 0
Fuel                    0
Residential             0
Surcharge               0
Freight Adj             0
Fuel Adj                0
Surcharge Adj           0
Duties/Tax              0
Entered Weight (LB)     0
Billed Weight (LB)      0
Dim Length             94
Dim Width              94
Dim Height             94
F+F+R                   0
Bill Amount             0
group_count             0
pair_count              0
dtype: int64

In [76]:
combined_weight_df = df_final.groupby(['Order Number', 'pair_count'], as_index=False)\
.agg({
    'Entered Weight (LB)': 'sum',
    'Dim Length': 'max',
    'Dim Width': 'max',
    'Dim Height': 'sum',
}).rename(columns={'Entered Weight (LB)': 'combine weight'})



In [ ]:
invoice_charge_df_sorted_grouped = invoice_charge_df.sort_values(by='Entered Weight (LB)', ascending=False).groupby('Order Number')

In [77]:
df_final[df_final['Order Number'].isin(['#1451227','#1448451'])]

,Order Number,Tracking Number,Origin State,Transaction Date,Receiver Zip,Carrier,Service Type,Freight,Fuel,Residential,Surcharge,Freight Adj,Fuel Adj,Surcharge Adj,Duties/Tax,Entered Weight (LB),Billed Weight (LB),Dim Length,Dim Width,Dim Height,F+F+R,Bill Amount,group_count,pair_count
954,#1448451,1ZAC83750306098167,TX,2025-03-03,294077607,UPS,Ground Residential,10.89,1.80,3.29,0.0,0.0,0.0,0.0,0.0,3.1,8.0,21.0,17.0,5.0,15.98,15.98,3,25.0
955,#1448451,1ZAC83750309295171,TX,2025-03-03,294077607,UPS,Ground Residential,10.89,1.80,3.29,0.0,0.0,0.0,0.0,0.0,3.1,8.0,21.0,17.0,5.0,15.98,15.98,3,25.0
956,#1448451,1ZAC83750319176585,TX,2025-03-03,294077607,UPS,Ground Residential,10.03,1.67,3.29,0.0,0.0,0.0,0.0,0.0,3.1,4.0,18.0,10.0,4.0,14.99,14.99,3,-
2542,#1451227,92346903255588300007010602,CA,2025-03-14,90049-3638,USPS,Ground Advantage,7.45,0.00,0.00,0.0,0.0,0.0,0.0,0.0,4.0,4.0,14.5,9.5,1.0,7.45,7.45,4,93.0
2543,#1451227,92346903255588300007010756,CA,2025-03-14,90049-3638,USPS,Ground Advantage,8.73,0.00,0.00,0.0,0.0,0.0,0.0,0.0,9.1,9.1,18.0,10.0,4.0,8.73,8.73,4,93.0
2544,#1451227,92346903255588300007010787,CA,2025-03-14,90049-3638,USPS,Ground Advantage,8.73,0.00,0.00,0.0,0.0,0.0,0.0,0.0,9.1,9.1,18.0,10.0,4.0,8.73,8.73,4,94.0
2545,#1451227,D10016218242139,CA,2025-03-14,90049-3638,OnTrac,C,13.75,1.39,0.88,0.0,0.0,0.0,0.0,0.0,11.0,40.0,20.0,14.0,10.0,16.02,16.02,4,94.0


In [78]:
combined_weight_df[combined_weight_df['Order Number'].isin(['#1451227','#1448451'])]

,Order Number,pair_count,combine weight,Dim Length,Dim Width,Dim Height
930,#1448451,25.0,6.2,21.0,17.0,10.0
931,#1448451,-,3.1,18.0,10.0,4.0
2450,#1451227,93.0,13.1,18.0,10.0,5.0
2451,#1451227,94.0,20.1,20.0,14.0,14.0
